### 수업 도서 복습/공부

## 02. 사이킷런으로 시작하는 머신러닝

#### 02 첫번째 머신러닝 만들어 보기 - 붓꽃 품종 예측하기
- 지도학습(Supervised Learning) > 분류(Classification)
    * 지도 학습 : 명확한 정답이 주어진 데이터를 먼저 학습한 뒤 미지의 정답을 예측

In [1]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

In [3]:
# DataFrame으로 데이터 구성 확인
import pandas as pd

# 붓꽃 데이터세트 로딩
iris = load_iris()

# iris.data : Iris 데이터 세트에서 피처(feature)만으로 된 데이터
iris_data = iris.data

# iris.target : Iris 데이터세트에서 레이블(결정값) 데이터
iris_label = iris.target
print('iris_target 값 :',iris_label)
print('iris target 이름 : ',iris.target_names)

# 붓꽃 데이터 세트 ==> DataFrame 변환
iris_df = pd.DataFrame(iris_data, columns=iris.feature_names)
# 라벨 열 추가
iris_df['label'] = iris.target
iris_df.head(3)

iris_target 값 : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
iris target 이름 :  ['setosa' 'versicolor' 'virginica']


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),label
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0


In [4]:
# 학습용 데이터와 테스트용 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(iris_data, iris_label, test_size=0.2, random_state=11)

In [5]:
# DecisionTreeClassifier 객체 생성
dt_clf = DecisionTreeClassifier(random_state=11)

In [6]:
# 학습 수행
dt_clf.fit(X_train, y_train)

# 학습이 완료된 DecisionTreeClassifier 객체에서 테스트 데이터 세트로 예측 수행
pred = dt_clf.predict(X_test)

In [7]:
# 예측된 결과를 기반으로 DecisionTreeClassifier의 예측 성능 평가
from sklearn.metrics import accuracy_score
print(f'예측 정확도 : {accuracy_score(y_test, pred):.4f}')

예측 정확도 : 0.9333


> 분류 예측 프로세스  
 : 데이터 세트 분리 -> 모델 학습 -> 예측 수행 -> 평가

#### 03 사이킷런 기반 프레임워크 익히기

- Estimator 클래스 : 지도학습의 모든 알고리즘을 구현한 클래스 (fit()과 predict()를 내부에서 구현)
    * Classifier : 분류 알고리즘을 구현한 클래스
    * Regressor  : 회귀 알고리즘을 구현한 클래스

- 비지도 학습을 구현한 클래스도 대부분 fit()과 transform()을 적용
    * 하지만 비지도학습과 피처 추출에서 fit()은 사전 구조를 맞추는 작업 / 학습을 의미하는 것이 아님

#### 04 Model Selection 모듈 소개

- 학습/테스트 데이터 세트 분리 - train_test_split()  
: 예측을 수행하는 데이터 세트는 학습을 수행한 학습용 데이터 세트가 아닌 전용의 테스트 데이터 세트여야 함

In [11]:
#- 교차 검증 : 데이터의 편중을 막기 위해 별도의 여러 세트로 구성된 학습 데이터 세트와 검증 데이터 세트에서 학습과 평가를 수행하는 것  
#    * ex) 과적합 : 모델이 학습 데이터에만 과도하게 최적화 -> 실제 예측을 다른 데이터로 수행할 경우 예측 성능이 과도하게 떨어지는 것

# - 교차 검증의 예시
#    * K폴드 교차 검증    : K개의 데이터 폴드 세트를 만들어 K번만큼 각 폴드 세트에 학습과 검증 평가를 반복적으로 수행 / K개의 예측 평가를 구했으면 그 평균을 결과로 반영  
#                        : split()을 호출하면 인덱스를 반환하기 때문에 실제 데이터 추출은 개발 코드에서 직접 수행 해야 함
#    * Stratified K 폴드 : 원본 데이터의 레이블 분포를 먼저 고려한 뒤 이 분포와 동일하게 학습과 검증 데이터 세트를 분배
#                        : split() 메서드 인자에 피처 데이터 세트 뿐만 아니라 레이블 데이터 세트도 반드시 필요함 (레이블 데이터 분포에 따라 학습/검증 데이터 나누기 때문)
#    * cross_val_score() : cv로 지정된 횟수만큼 scoring 파라미터로 지정된 평가 지표로 평가 결괏값을 배열로 반환 / 이를 평균해 평가 수치로 사용
#    * GridSearchCV      : 데이터 세트를 cross-validation을 위한 학습/테스트 세트로 자동 분할 -> 하이퍼 파라이터 그리드에 기술된 모든 파라미터를 순차적으로 적용해 최적의 파라미터 찾음

#### 05 데이터 전처리

- 데이터 인코딩
    * 사이킷런의 머신러닝 알고리즘은 문자열 값을 입력값으로 허용X -> 모든 문자열 값을 인코딩 시켜 숫자형으로 변환해야함

- 데이터 인코딩 종류
    * 레이블 인코딩 : 카테고리 피처를 코드형 숫자값으로 변환
    * 원-핫 인코딩 : 행 형태로 돼 있는 피처의 고유값을 열 형태로 차원을 변환한 뒤 고유값에 해당하는 칼럼에만 1을 표시하고 나머지 칼럼에는 0을 표시 (Pandas - get_dummies(df))

- 피처 스케일링 : 서로 다른 변수의 값 범위를 일정한 수준으로 맞추는 작업 / 예) 표준화, 정규화
    * 표준화 : 데이터 피처가 가우시안 정규분포를 가진 값으로 변환하는 것 (평균 0, 분산 1)
    * 정규화 : 서로 다른 피처의 크기를 통일하기 위해 크기를 변환해 주는 것

- StandardScaler    
 : 표준화를 지원해주는 클래스  
 : 서포트벡터머신, 선형회귀, 로지스틱회귀는 데이터가 가우시안 분포를 가지고 있다고 가정하고 구현됨 -> 사전에 표준화를 적용해야 예측 성능이 향상  
  
- MinMaxScaler  
 : 데이터값을 0과 1사이의 범위 값으로 변환 (음수가 있으면 -1에서 1 값으로 변환)
 : 데이터의 분포가 가우시간 분포가 아닐 경우 적용 가능

- 학습 데이터와 테스트 데이터의 스케일링 변환 시 유의점
 * fit() : 데이터 변환을 위한 기준 정보 설정
 * transform() : 설정된 정보를 이용해 데이터 변환
 * 학습 데이터로 fit()이 적용된 스케일링 기준 정보를 그대로 테스트 데이터에 적용해야함!!  
  
 * 가능하다면 전체 데이터의 스케일링 변환을 적용한 뒤 학습과 테스트 데이터로 분리
 * 위가 불가능 하면 테스트 데이터 변환 시에는 fit()이나 fit_transform()을 적용하지 않고 학습 데이터로 이미 fit()된 Scaler 객체를 이용해 transform()으로 변환

#### 07 정리

- 머신러닝 애플리 케이션의 단계
    * 데이터 전처리(데이터의 가공및 변환 과정) -> 데이터 세트 분리(데이터를 학습데이터와 테스트데이터로 분리) -> 모델 학습(학습데이터를 기반으로 머신러닝 알고리즘 적용) ->   
    예측 수행(학습된 모델을 기반으로 한 테스트 데이터) -> 머신러닝 모델 평가(예측된 결괏값을 실제 결괏값과 비교)